## Jaccard Similarity
Jaccard similarity is defined between two sets as the ratio of the volume of their intersection divided by the volume of their union. In the context of graphs, the neighborhood of a vertex is seen as a set. The Jaccard similarity weight of each edge represents the strength of connection between vertices based on the relative similarity of their neighbors. For further detail see [Wikipedia](https://en.wikipedia.org/wiki/Jaccard_index)

To Compute the Jaccard similarity between each pair of vertices connected by an edge in cuGraph use: 
**nvJaccard(G)**
* G: A cugraph.Graph object

Returns:
jaccard_weights: cudf.DataFrame with three columns:
* jaccard_weights['source']: The source vertex id.
* jaccard_weights['destination']: The destination vertex id.
* jaccard_weights['jaccard_coeff']: The jaccard coefficient computed between the source and destination vertex.

In [1]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

import numpy as np
from scipy.io import mmread

In [2]:
# define a function for printing out the top most similar vertices
def printMostSimilar(df):
    bestEdge = 0
    
    for i in range(len(df)):
        if df['jaccard_coeff'][i] > df['jaccard_coeff'][bestEdge]:
            bestEdge = i
        
    print("Vertices " + str(df['source'][bestEdge]) + " and " + 
          str(df['destination'][bestEdge]) + " are most similar with score: " 
          + str(df['jaccard_coeff'][bestEdge]))

## Reading a CSV file using cuDF

In [3]:
# Test file  - using the clasic Karate club dataset.  
datafile='../data/networks/karate-data.csv'

In [4]:
# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()), )

In [5]:
# create a Graph 
G1 = cugraph.Graph()
G1.add_edge_list(gdf["src"], gdf["dst"])

In [6]:
# How many vertices are in the graph?
G1.num_vertices()

35

In [7]:
# Call cugraph.nvJaccard 
df1 = cugraph.nvJaccard(G1)

In [8]:
printMostSimilar(df1)

Vertices 33 and 34 are most similar with score: 0.5555556


In [16]:
G1.view_edge_list()

(<cudf.Series nrows=154 >, <cudf.Series nrows=154 >)

## As a Matrix file using scipy

In [29]:
# Read in the data file into scipy matrix format
mmFile='../data/networks/karate.mtx'
M = mmread(mmFile).asfptype().tolil()
M = M.tocsr()

In [30]:
# Load the structure of the graph into GPU memory and create a CuGraph
# graph object:
row_offsets = cudf.Series(M.indptr)
col_indices = cudf.Series(M.indices)
values = cudf.Series(M.data)
G2 = cugraph.Graph()
G2.add_adj_list(row_offsets, col_indices, values)

In [31]:
# How many vertices are in the graph?
G2.num_vertices()

34

In [32]:
G2.view_edge_list()

(<cudf.Series nrows=156 >, <cudf.Series nrows=156 >)

In [33]:
# Call cugraph.nvJaccard 
df2 = cugraph.nvJaccard(G2)

In [34]:
printMostSimilar(df2)

Vertices 15 and 33 are most similar with score: 2.6271076e+30


___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___